# Erwartungswert

In [1]:
from qib.operator import FieldOperator, FieldOperatorTerm, IFOType, IFODesc
import fermitensor as ftn # after installing fermitensor (installed with kernel 3.10.11)

import hamiltonian_eq_9 as ham
import hamiltonian_eq_9_tests as ham_test
import matrix_reference as ref

import numpy as np
# from scipy import sparse
from scipy.sparse.linalg import norm

### Hamiltonian for TTN

In [2]:
L = 8
L_1 = L//2
A = range(L_1)
B = range(L_1, L)

# latt = qib.lattice.FullyConnectedLattice((L,))
# field = qib.field.Field(qib.field.ParticleType.FERMION, latt)
# tkin, vint = construct_random_coefficients(L)

# create MolecularHamiltonian Object
H_ref = ref.construct_random_molecular_hamiltonian(L)
field = H_ref.field
tkin = H_ref.tkin
vint = H_ref.vint


#### Create all components in (11)-(13)

In [3]:
# final Hamiltonian according to equ. (9)
H_A = ham.get_part_of_H_as_FO(field, tkin, vint, A)
H_B = ham.get_part_of_H_as_FO(field, tkin, vint, B)

In [4]:
H_AB = ham.get_H_AB_as_FO(field, tkin, vint, A, B)

In [5]:
def csr_allclose(a, b, rtol=1e-6, atol = 1e-8):
    c = np.abs(a - b)
    help = np.round(c,5)
    help.eliminate_zeros()
    print(help)
    # print(help.max())
    return c.max() <= atol + rtol * np.abs(b)

In [6]:
H = H_A + H_B + H_AB
# H = H_A + H_B

In [9]:
ham_test.is_hermitian(H)

True

In [7]:
# compare new hamiltonian with other
# norm der differenz sollte 0 sein
norm(H.as_matrix()- H_ref.as_matrix())
# assert csr_allclose(H.as_matrix(), H_ref.as_matrix(), atol=1e-5)

59.682485299395246